In [37]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [38]:
### Load the trained model, scaler pickle, onehot
model = load_model('mode.h5')

# laod the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

In [ ]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',   
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [40]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df
 

/Users/piotrzurkowski/repo/my_ann_classification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [41]:
input_df = pd.DataFrame([input_data])

In [42]:
#Encode categorical values
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])

In [43]:
## concatination one hot encoded
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df.drop('Geography', axis=1, inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [44]:
# Scaling input data
input_scaled = scaler.transform(input_df)

In [45]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [46]:
# Prediction
prediction = model.predict(input_scaled)
prediction

2026-02-19 00:03:47.232910: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step


array([[0.06133188]], dtype=float32)

In [47]:
prediction_propability = prediction[0][0]

In [48]:
prediction_propability


0.061331883